In [ ]:
import duckdb
import geopandas as gpd
import pandas as pd
from h3pandas.util.shapely import polyfill
import pyarrow.dataset as ds

In [ ]:
wkt = "POLYGON ((-48.251953 -2.723583, -48.251953 3.951941, -37.089844 3.951941, -37.089844 -2.723583, -48.251953 -2.723583))"

In [ ]:
con = duckdb.connect()
con.sql("""
    INSTALL spatial;
    LOAD spatial;
""")

dataset = ds.dataset("../h3_7/", format="parquet")
con.register("dataset", dataset)

In [ ]:
df = con.execute(f"""
	select
        species,
        sum(records) as records,
        min(min_year) as min_year,
        max(max_year) as max_year,
        max(source_obis) as source_obis,
        max(source_gbif) as source_gbif
    from dataset
    where ST_Intersects(ST_GEOMFROMWKB(geometry), ST_GeomFromText('{wkt}')) 
    group by species
""").fetchdf()
df